# Event Display Generator

_Authors: Jurjan Bootsma, Bouke Jung, Maarten de Jong_

This jupyter notebook contains a Monte-Carlo event generator for incoming neutrinos reacting with water. You can choose at which event you want to look, at what flavour you want to look (normal or anti-neutrino) and if you want to look at low or high energies. Then you can look at all kinds of plots, namely a plot of the directions, of all the momentums of the particles (looking from the neutrino), a polar plot $\phi$ against $\theta$ and a polar plot with $\phi$ against the transverse momentum. In these plots the neutrino and the outgoing lepton are marked with another colour than the other particles.

In [3]:
import matplotlib.pyplot as plt
import numpy as np

import km3pipe as kp
import km3modules as km
import km3io
from km3net_testdata import data_path
from particle import Particle
import ipywidgets as iw
import pylorentz as pl
from km3pipe import bootsma

%matplotlib widget

In [4]:
file_electron_low =  "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_elec-CC_1-100GeV.135.root"
file_anti_electron_low = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_anti-elec-CC_1-100GeV.106.root"
file_muon_low = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_muon-CC_1-100GeV.111.root"
file_anti_muon_low = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_anti-muon-CC_1-100GeV.136.root"
file_tau_low = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_tau-CC_3-100GeV.100.root"
file_anti_tau_low = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_anti-tau-CC_3-100GeV.1014.root"

file_electron_high = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_elec-CC_100-500GeV.133.root"
file_anti_electron_high = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_anti-elec-CC_100-500GeV.107.root" 
file_muon_high = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_muon-CC_100-500GeV.2.root"
file_anti_muon_high = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_anti-muon-CC_100-500GeV.127.root"
file_tau_high = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_tau-CC_100-300GeV.172.root"
file_anti_tau_high = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_anti-tau-CC_100-300GeV.132.root"

# Functions

In [5]:
def dir_plotter_3D_arrow(f_event):
    pos = tuple(np.zeros(f_event.n_mc_trks))
    dir_x = tuple(f_event.mc_tracks.dir_x)
    dir_y = tuple(f_event.mc_tracks.dir_y)
    dir_z = tuple(f_event.mc_tracks.dir_z)
    
    lepton_bool = bootsma.lepton_finder(f_event.mc_tracks.pdgid)
    lepton_bool2 = ~ lepton_bool
    
    fig, ax = plt.subplots(figsize=(7.5,7.5))
    ax = plt.axes(projection='3d')
    ax.quiver(0, 0, 0, dir_x[0], dir_y[0], dir_z[0], color='blue', label='incoming neutrino')
    ax.quiver(0, 0, 0, lepton_bool[1:]*dir_x[1:], lepton_bool[1:]*dir_y[1:], lepton_bool[1:]*dir_z[1:], color='green', label='outgoing lepton')
    ax.quiver(0, 0, 0, lepton_bool2[1:]*dir_x[1:], lepton_bool2[1:]*dir_y[1:], lepton_bool2[1:]*dir_z[1:], color='red', label='other particles')
    ax.set_xlabel('dx')
    ax.set_ylabel('dy')
    ax.set_zlabel('dz')
    ax.set_xlim(-1,1)
    ax.set_ylim(-1,1)
    ax.set_zlim(-1,1)
    ax.legend()
    ax.set_title("Event: Incoming Electron-neutrino")
    plt.show()

In [6]:
def lepton_finder(pdg_ids):
    electron_bool = (np.abs(pdg_ids) == 11)
    muon_bool = (np.abs(pdg_ids) == 13)
    tau_bool = (np.abs(pdg_ids) == 15)
    lepton_bool = np.array(electron_bool+muon_bool+tau_bool)
    return lepton_bool

In [7]:
def filename_finder(channel,energy):
    if energy == "low":
        if channel == "Electron":
            filename = file_electron_low
        elif channel == "Muon":
            filename = file_muon_low
        elif channel == "Tau":
            filename = file_tau_low
        elif channel == "Anti-Electron":
            filename = file_anti_electron_low
        elif channel == "Anti-Muon":
            filename = file_anti_muon_low
        elif channel == "Anti-tau":
            filename = file_anti_tau_low
    elif energy == "high":
        if channel == "Electron":
            filename = file_electron_high
        elif channel == "Muon":
            filename = file_muon_high
        elif channel == "Tau":
            filename = file_tau_high
        elif channel == "Anti-Electron":
            filename = file_anti_electron_high
        elif channel == "Anti-Muon":
            filename = file_anti_muon_high
        elif channel == "Anti-tau":
            filename = file_anti_tau_high
    return filename

In [8]:
def particles_momentum(f_event):
    vectors_final = rotation_particles(f_event)
    
    momentum = get_momentum(f_event.mc_tracks)
    
    vectors_final[0] = vectors_final[0]*momentum
    vectors_final[1] = vectors_final[1]*momentum
    vectors_final[2] = vectors_final[2]*momentum
    
    return vectors_final

In [9]:
def polar(x, y, z):
    r = np.sqrt(x**2+y**2+z**2)
    theta = np.arccos(z/r)
    theta = np.nan_to_num(theta)
    
    y_mask1 = (y>=0)
    
    phi1 = np.arccos(x/np.sqrt(x**2+y**2))
    phi_array1 = phi1*y_mask1
       
    y_mask2 = (y<0)
    phi2 = 2*np.pi - np.arccos(x/np.sqrt(x**2+y**2))
    phi_array2 = phi2*y_mask2
    
    phi = phi_array1+phi_array2
    phi = np.nan_to_num(phi)
    
    return r, theta, phi

In [10]:
def rotation_particles(f_event):
    pos = tuple(np.zeros(f_event.n_mc_trks))
    dir_x = tuple(f_event.mc_tracks.dir_x)
    dir_y = tuple(f_event.mc_tracks.dir_y)
    dir_z = tuple(f_event.mc_tracks.dir_z)
    
    vector = np.array([dir_x[0], dir_y[0], dir_z[0]])
    
    vector_without_y = np.array([vector[0], 0, vector[2]])    
    angle_from_z = kp.math.angle(vector_without_y, np.array([1,0,0]))
    y_axis = np.array([0,1,0])
    
    if vector[2] < 0:
        angle_from_z = (2*np.pi-angle_from_z)
    rotation_matrix_from_z = kp.math.rotation_matrix(y_axis,angle_from_z)
    
    vector_new = rotation_matrix_from_z@vector
    
    vector_without_z = np.array([vector_new[0], vector_new[1], 0])
    angle_from_y = kp.math.angle(vector_without_z, np.array([1,0,0]))
    z_axis = np.array([0,0,1])
    
    if vector_new[1] > 0:
        angle_from_y = (2*np.pi-angle_from_y)
    rotation_matrix_from_y = kp.math.rotation_matrix(z_axis,angle_from_y)
    
    vector_final = rotation_matrix_from_y@vector_new
    
    vectors = np.array([dir_x, dir_y, dir_z])
    vectors_new = rotation_matrix_from_z@vectors
    vectors_final = rotation_matrix_from_y@vectors_new
    
    return vectors_final

In [11]:
def get_momentum(mc_tracks):
    masses = get_masses(mc_tracks) #MeV
    Es = mc_tracks.E #GeV
    Es_MeV = Es * 10**3 #MeV
    momentum = np.sqrt(Es_MeV**2 - masses**2)
    momentum = np.nan_to_num(momentum) * 10**-3
    return momentum

In [12]:
def get_masses(mc_tracks):
    pdg_ids = mc_tracks.pdgid
    masses = np.zeros(len(pdg_ids))
    i=0
    for pdg in pdg_ids:
        p = Particle.from_pdgid(pdg)
        mass = p.mass
        if mass == None:
            mass = 0
        masses[i]=mass
        i=i+1
    return masses

# Generator

In [13]:
opts = dict(continuous_update=False)
@iw.interact
def interactive_generator(event=iw.BoundedIntText(value=0,min=0,max=100,description="Event"),
                          channel=iw.Dropdown(options=[("Electron","Electron"),("Muon","Muon"),
                                                       ("Tau","Tau"),("Anti-Electron","Anti-Electron"),
                                                       ("Anti-Muon","Anti-Muon"),("Anti-Tau","Anti-Tau")],
                                              description="Neutrino"),
                          energy_range=iw.Dropdown(options=[("<100 GeV","low"),(">100 GeV","high")],
                                                   description="Energy"),
                          plot=iw.Dropdown(options=[("Directions","directions"),("Momentum","momentum"),
                                                    ("Polar plot transverse p", "p_t"),],
                                           description="Kind of plot")):
    
    filename = filename_finder(channel, energy_range)
    f = km3io.OfflineReader(data_path(filename))        
                          
    energies = f[event].mc_tracks.E
    lepton_plot = (filename == file_electron_low or filename == file_muon_low\
                   or filename == file_anti_electron_low or filename == file_anti_muon_low\
                   or filename == file_electron_high or filename == file_muon_high\
                   or filename == file_anti_electron_high or filename == file_anti_muon_high)
    if lepton_plot == True:
        lepton_bool = lepton_finder(f[event].mc_tracks.pdgid)
        lepton_bool2 = ~ lepton_bool
        lepton_index = np.where(lepton_bool)[0][0]
    if plot == "directions":
        dir_plotter_3D_arrow(f[event])
    elif plot == "momentum" or plot=="p_t":
        momentum = particles_momentum(f[event])
        if plot == "momentum":
            fig, ax = plt.subplots(figsize=(7.5,7.5))
            ax = plt.axes(projection='3d')
            ax.quiver(0, 0, 0, momentum[0][0], momentum[1][0], momentum[2][0], arrow_length_ratio = 0.0, color='blue',label=f"Neutrino {round(energies[0],1)} GeV")
            if lepton_plot==True:
                ax.quiver(0, 0, 0, lepton_bool[1:]*momentum[0][1:], lepton_bool[1:]*momentum[1][1:], lepton_bool[1:]*momentum[2][1:], arrow_length_ratio = 0.0, color='green',label=f"Lepton {round(energies[lepton_index],1)} GeV")
                ax.quiver(0, 0, 0, lepton_bool2[1:]*momentum[0][1:], lepton_bool2[1:]*momentum[1][1:], lepton_bool2[1:]*momentum[2][1:], arrow_length_ratio = 0.0, color='red', label="Particles")
            elif lepton_plot==False:
                ax.quiver(0, 0, 0, momentum[0][1:], momentum[1][1:], momentum[2][1:], arrow_length_ratio = 0.0, color='red',label="Particles")
            ax.set_xlabel('$p_x$')
            ax.set_ylabel('$p_y$')
            ax.set_zlabel('$p_z$')
            ax.set_xlim(-np.max(np.abs(momentum[0])),np.max(np.abs(momentum[0])))
            ax.set_ylim(-np.max(np.abs(momentum[1])),np.max(np.abs(momentum[1])))
            ax.set_zlim(-np.max(np.abs(momentum[2])),np.max(np.abs(momentum[2])))
            ax.view_init(0,180)
            plt.legend(bbox_to_anchor=(1, 0.85), loc="right")
            plt.show()            
        elif plot == "p_t":
            r_mom, theta_mom, phi_mom = polar(momentum[1],momentum[2],momentum[0])
            momentum_t = np.sqrt(momentum[1]**2 + momentum[2]**2)
            fig = plt.figure(figsize=(7.5,7.5))
            ax = fig.add_subplot(projection='polar')
            c1 = ax.scatter(phi_mom[0], momentum_t[0], color='blue',label=f"Neutrino {round(energies[0],1)} GeV")
            if lepton_plot==True:
                c3 = ax.scatter(phi_mom[lepton_index], momentum_t[lepton_index], color="green",label=f"Lepton {round(energies[lepton_index],1)} GeV")
            c2 = ax.scatter(np.delete(phi_mom,1), np.delete(momentum_t,1),color='red',label="Particles")
            plt.legend(bbox_to_anchor=(1.35, 1), loc="right")
            plt.show()                

            

interactive(children=(BoundedIntText(value=0, description='Event'), Dropdown(description='Neutrino', options=(…